In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from einops import rearrange, repeat, reduce
import serial
import scipy
from IPython.display import HTML
import time
from tqdm.notebook import tqdm
import pyvisa

np.set_printoptions(precision=4, suppress=True)

In [3]:
def open_device():
    global device
    if 'device' in globals():
        if device is not None:
            device.close()
    rm = pyvisa.ResourceManager()
    devices = rm.list_resources()
    for d in devices:
        if d.startswith("USB0"):
            device = rm.open_resource(d)
            print("Device found: ", d)
            print(device.query("*IDN?"))
            return
    raise Exception("No device found")

In [4]:
# Connect to Siglent SDG2122X
open_device()

Device found:  USB0::0xF4EC::0x1102::SDG2XBA4150426::INSTR
Siglent Technologies,SDG2122X,SDG2XBA4150426,2.01.01.37R6



In [27]:
def setup_device():
    device.write("*RST")
    time.sleep(0.1)
    device.write("C1:OUTP LOAD,50")
    device.write("C1:BSWV FRQ,2000000")
    device.write("C1:BSWV AMP,0.1")
    device.write("C1:OUTP ON")
    # device.flush()
    # time.sleep(2)

# setup_device()

In [32]:
def set_carrier():
    device.write("C1:MDWV STATE,OFF")


def set_am(am_freq, am_depth):
    device.write("C1:MDWV STATE,ON")
    # time.sleep(1)
    device.write("C1:MDWV AM")
    device.write(f"C1:MDWV AM,MDSP,SINE")
    device.write(f"C1:MDWV AM,FRQ,{am_freq}")
    device.write(f"C1:MDWV AM,DEPTH,{am_depth * 100}")


def set_fm(fm_freq, fm_dev):
    device.write("C1:MDWV STATE,ON")
    # time.sleep(1)
    device.write("C1:MDWV FM")
    device.write(f"C1:MDWV FM,MDSP,SINE")
    device.write(f"C1:MDWV FM,FRQ,{fm_freq}")
    device.write(f"C1:MDWV FM,DEV,{fm_dev}")


def set_ask(am_freq, am_depth):
    device.write("C1:MDWV STATE,ON")
    # time.sleep(1)
    device.write("C1:MDWV AM")
    device.write(f"C1:MDWV AM,MDSP,SQUARE")
    device.write(f"C1:MDWV AM,FRQ,{am_freq}")
    device.write(f"C1:MDWV AM,DEPTH,{am_depth * 100}")


def set_fsk(fm_freq, fm_hop):
    device.write("C1:MDWV STATE,ON")
    # time.sleep(1)
    device.write("C1:MDWV FSK")
    device.write(f"C1:MDWV FSK,KFRQ,{fm_freq}")
    device.write(f"C1:MDWV FSK,HFRQ,{fm_hop}")

In [31]:
# setup_device()
set_fsk(8_000, 2_000_100)